# Look-Locker T1-maps
This notebook attempts to describe the procedure of creating $T_1$-maps from a Look-Locker inversion recovery (IR) sequence, for the purpose of tracer concentration estimation in brain tissue and cerebrospinal fluid (CSF).

The steps include:
- Converting necessary volumes from enhanced DICOM to Nifti.
- Estimating $T_1$-times in each voxel by fitting a curve to the IR-signal, followed by a Look-Locker correction.
- Registration of $T_1$-maps to the reference space of a pre-contrast $T_1$-weighted image.

## Setup
### Dependencies:
- FreeSurfer (v7.4.1)
- Greedy

### Data
To generate the $T_1$-maps from Look-Locker images you need the following data for each subject, (with `ses` representing the session):
- `mri_dataset/$subject/ses-01/anat/${subject}_ses-01_T1w.nii.gz`
- `mri_dataset/$subject/$ses/anat/${subject}_${ses}_acq-looklocker_IRT1.nii.gz`
- `mri_dataset/$subject/$ses/mixed/${subject}_${ses}_acq-looklocker_IRT1_trigger_times.txt`

The  `mri_dataset/$subject/$ses/mixed/${subject}_${ses}_acq-looklocker_IRT1_trigger_times.txt`-files contain trigger times in milli-seconds for each of the images in the inversion-recovery sequence, and are required for curve-fitting in the $T_1$-map generation, and are created as part of the DICOM-conversion process.

### Raw $T_1$-map generation
The $T_1$-maps are generated from the Look-Locker sequence by a voxelwise curve-fitting to the IR signal, as further described in the [accompanying article](https://doi.org/10.1101/2025.07.23.25331971), or in the notebook [ (fill inn path to detailed LL-notebook )]().
The command to generate a Look-Locker $T_1$-map is given below.
Note that this script will typically take somewhere from 40 minutes up to a couple of hours depending on your system, due to the shear number of voxels in high-resolution 3D images.
```bash
gmri2fem mri looklocker-t1map \
    --input mri_dataset/$subject/$ses/anat/${subect}_${ses}_acq-looklocker_IRT1.nii.gz \
    --timestamps mri_dataset/$subject/$ses/anat/${subect}_${ses}_acq-looklocker_IRT1.nii.gz \
    --output mri_dataset/derivatives/$subject/$ses/${subject}_${ses}_acq-looklocker_T1map_raw.nii.gz
```

### Post-processing $T_1$-maps
The raw $T_1$-maps generated above likely includes a number of undefined (NaN)-values, or a number of voxels with unphysiologically low or high voxel values.
This primarily happens for voxels with either a very short or very large $T_1$-time compared to the parenchyma, but might ocationally occur within the brain tissue itself.

These voxels are imputed from its surroundings,

```bash
gmri2fem mri looklocker-t1-postprocessing \
    --LL mri_dataset/sub-01/ses-01/anat/sub-01_ses-01_acq-looklocker_IRT1.nii.gz \
    --T1map mri_dataset/derivatives/sub-01/ses-01/sub-01_ses-01_acq-looklocker_T1map_raw.nii.gz \
    --output mri_dataset/derivatives/sub-01/ses-01/sub-01_ses-01_acq-looklocker_T1map.nii.gz \
    --T1_low 100 \
    --T1_high 6000
```

### $R_1$-map generation

```bash
gmri2fem mri t1-to-r1 \
    --input mri_dataset/derivatives/sub-01/ses-01/sub-01_ses-01_acq-looklocker_T1map.nii.gz \
    --output mri_dataset/derivatives/sub-01/ses-01/sub-01_ses-01_acq-looklocker_R1map.nii.gz \
    --T1_low 100 \
    --T1_high 6000
```

### Registration
Registration
```bash
greedy -d 3 -a \
    -i mri_processed_data/sub-01/registered/sub-01_ses-01_T1w_registered.nii.gz mri_dataset/derivatives/sub-01/ses-01/sub-01_ses-01_acq-looklocker_R1map.nii.gz \
    -o mri_processed_data/sub-01/transforms/sub-01_ses-01_acq-looklocker.mat \
    -ia-image-centers \
    -dof 6 \
    -m NCC 5x5x5 \
    -threads 6
```
Reslicing
```bash
greedy -d 3 -rf \
    mri_processed_data/sub-01/registered/sub-01_ses-01_T1w_registered.nii.gz \
    -ri NN \
    -rm mri_dataset/derivatives/sub-01/ses-01/sub-01_ses-01_acq-looklocker_T1map.nii.gz mri_processed_data/sub-01/registered/sub-01_ses-01_acq-looklocker_T1map_registered.nii.gz \
    -r mri_processed_data/sub-01/transforms/sub-01_ses-01_acq-looklocker.mat \
    -threads 4
```